In [1]:
from bs4 import BeautifulSoup
import re
import pandas as pd
import yfinance as yf

In [43]:
# get product id & product url from downloaded ishare ETF html file
# this is for downloading up-to-date holdings file from ishare
website_header = "https://www.ishares.com"

html = open("product_id_ishare.html", encoding="utf-8").read()
soup = BeautifulSoup(html, "html.parser")

rows = soup.select("tr:has(td.links a[href*='/produkte/'])")

records = []

for row in rows:
    links = row.select("td.links a")

    if len(links) >= 2:
        product_url = links[0].get("href")

        ids = [re.search(r"/produkte/(\d+)/", a["href"]).group(1) for a in links[:2]]

        id1, id2 = ids
        ticker = links[0].get_text(strip=True)
        fund_name = links[1].get_text(strip=True)
        records.append(
            {
                "product_id": id2,
                'product_url':website_header + product_url,
                "ticker": ticker,
                "fund_name": fund_name,
                "flag": id1 == id2,
            }
        )
records_df = pd.DataFrame(records)
records_df['ticker']=records_df['ticker'].apply(lambda x: x.strip())

In [3]:
# load selected ETF dataframe
ETF_file = pd.read_excel("iShares-Germany_filtered_ETF_descending.xlsx")
mask_etf = ETF_file["Fondstyp"] == "ETF"
mask_asset_class = ETF_file["Anlageklasse"] == "Aktien"
filtered_etf = (
    ETF_file[mask_etf & mask_asset_class]
    .reset_index()
    .sort_values(by="Fondsvermögen", ascending=False)
)
# select top 120
filtered_etf=filtered_etf.head(120)
filtered_etf['Ticker'] = filtered_etf['Ticker'].apply(lambda x: x.strip())

tickers = filtered_etf['Ticker'].tolist()
tickers = [ticker.strip() for ticker in tickers]


In [4]:
# prepare ticker names for yfinance price download
suffixes = [".DE", ".L", ".SW", ".MI", ".PA", ".AS"]

def find_valid_ticker(base):
    for s in suffixes:
        t = base + s
        data = yf.download(t, period="5d", progress=False)
        if not data.empty:
            return t
    return None

valid_map = {}
invalid = []

for t in tickers:
    if t.__contains__('.'):
        real = t
    else:
        real = find_valid_ticker(t)

    if real:
        valid_map[t] = real
    else:
        invalid.append(t)

print(valid_map)
print("still invalid:", invalid)

C:\Users\lizhiqian\AppData\Local\Temp\ipykernel_24040\3396949129.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(t, period="5d", progress=False)
C:\Users\lizhiqian\AppData\Local\Temp\ipykernel_24040\3396949129.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(t, period="5d", progress=False)
C:\Users\lizhiqian\AppData\Local\Temp\ipykernel_24040\3396949129.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(t, period="5d", progress=False)
C:\Users\lizhiqian\AppData\Local\Temp\ipykernel_24040\3396949129.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(t, period="5d", progress=False)
C:\Users\lizhiqian\AppData\Local\Temp\ipykernel_24040\3396949129.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(t, period="5d", progre

{'SXR8': 'SXR8.DE', 'EUNL': 'EUNL.DE', 'SXRZ': 'SXRZ.DE', 'EXX7': 'EXX7.DE', 'IS3N': 'IS3N.DE', 'IUSQ': 'IUSQ.DE', 'SXRV': 'SXRV.DE', 'IUSA': 'IUSA.DE', 'SGAS': 'SGAS.DE', 'OM3L': 'OM3L.DE', 'QDVE': 'QDVE.DE', 'EUNK': 'EUNK.DE', 'IUSZ': 'IUSZ.DE', 'I500': 'I500.DE', 'IQQY': 'IQQY.DE', 'EXSA': 'EXSA.DE', 'IQQW': 'IQQW.DE', 'DAXEX': 'DAXEX.SW', 'EXW1': 'EXW1.DE', 'EDMU': 'EDMU.DE', 'IBCF': 'IBCF.DE', 'EDM2': 'EDM2.DE', 'IQQE': 'IQQE.DE', 'IUSN': 'IUSN.DE', 'EUNN': 'EUNN.DE', 'SXRT': 'SXRT.DE', 'EUNM': 'EUNM.DE', '2B7K': '2B7K.DE', 'SXR7': 'SXR7.DE', 'EXXT': 'EXXT.DE', 'AYEM': 'AYEM.DE', 'NDIA': 'NDIA.L', 'CEBL': 'CEBL.DE', 'SNAW': 'SNAW.DE', 'EDMW': 'EDMW.DE', 'EUN2': 'EUN2.DE', 'IS3S': 'IS3S.DE', 'IBCH': 'IBCH.DE', 'IS0E': 'IS0E.DE', 'SLMC': 'SLMC.DE', 'IS3Q': 'IS3Q.DE', 'ICGA': 'ICGA.DE', 'EMNU': 'EMNU.DE', '84X0': '84X0.DE', 'O4J0': 'O4J0.DE', 'ISPA': 'ISPA.DE', 'IS3R': 'IS3R.DE', '2B76': '2B76.DE', 'EXV1': 'EXV1.DE', 'QDVS': 'QDVS.DE', 'SXR1': 'SXR1.DE', '4BRZ': '4BRZ.DE', 'SXR4': 'S

In [5]:
# download prices from yfinance
valid_tickers = list(valid_map.values())
data = yf.download(valid_tickers, period="3y")

if isinstance(data.columns, pd.MultiIndex):
    adj_close = data["Close"]
else:
    adj_close = data[["Close"]]  # For single ticker cases

C:\Users\lizhiqian\AppData\Local\Temp\ipykernel_24040\761858267.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(valid_tickers, period="3y")
[*********************100%***********************]  120 of 120 completed


In [ ]:
# save prices
prices_df = adj_close.reset_index().melt(id_vars=['Date'])
prices_df.to_csv('prices.csv')


In [44]:
# check if all selected tickers are available in the downloaded html ishare file
manual_list = []
for tick in tickers:
    if tick not in records_df['ticker'].tolist():
        print(f'???{tick} not found')
        manual_list.append(tick)

print(manual_list)
print(len(manual_list))

[]
0


In [45]:
records_df.drop_duplicates(subset=['ticker'],inplace=True)

In [46]:
# remove irrelevant tickers
records_df = records_df[records_df['ticker'].isin(filtered_etf['Ticker'])]

In [47]:
len(records_df) 

120

In [ ]:
import re
import utils_webscrap


xls_file_url_list = []
for row in records:
    final_href = utils_webscrap.generate_ishares_download_link(row['product_url'], row['fund_name'],row['ticker'])
    # print(f"Generated URL: {final_href}")
    xls_file_url_list.append({'download_url': final_href,'product_id':row['product_id']})
# extend records_df by product download url
records_df = pd.merge(pd.DataFrame(xls_file_url_list),records_df,how='right',on='product_id')

In [58]:
import xml.etree.ElementTree as ET
import pandas as pd
from io import StringIO


def parse_holdings_xls(xml_path):
    # ===== 1. read in file =====
    with open(xml_path, "r", encoding="utf-8", errors="ignore") as f:
        content = f.read()

    # ===== 2. XML =====
    xml_start = content.find("<?xml")
    if xml_start == -1:
        raise ValueError("XML not found in file")

    clean_xml = content[xml_start:]
    clean_xml = clean_xml.replace("&", "&amp;")
    clean_xml = clean_xml.replace("&amp;amp;", "&amp;")
    # ===== 3. decode XML =====
    root = ET.fromstring(clean_xml)

    ns = {"ss": "urn:schemas-microsoft-com:office:spreadsheet"}

    table = root.find(".//ss:Worksheet/ss:Table", ns)
    rows = table.findall("ss:Row", ns)

    header = None
    data = []
    start = False

    for row in rows:
        cells = row.findall("ss:Cell", ns)
        values = []

        for cell in cells:
            d = cell.find("ss:Data", ns)
            values.append(d.text if d is not None else None)

        # get table header
        if values and "Emittententicker" in values:
            header = values
            start = True
            continue

        if start and any(v not in (None, "") for v in values):
            data.append(values)

    df = pd.DataFrame(data, columns=header)

    # print(df.head())
    print(f"\nsuccessfully extracted {len(df)} rows")
    return df
    

In [ ]:
import requests
import random
import time

master_industry_list = []
master_location_list = []
df = []
manual_inspect =[]


def retrieve_holding_info_from_ishare(download_url, referer_url, ticker):

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
        "Referer": referer_url,  # we are from your own website, plz dont block us :[
        "Accept": "text/csv,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
    }

    # use session to avoid 403 blockage
    session = requests.Session()

    try:
        response = session.get(download_url, headers=headers)
        time.sleep(random.randint(1,3))
        if response.status_code == 403:
            print(f"status code: {response.status_code} for ticker {ticker}")
            manual_inspect.append(ticker)

        if response.status_code == 200:
            with open("holdings_info.xls", "wb") as f:
                f.write(response.content)
            holdings_df = parse_holdings_xls("holdings_info.xls")
            df.append(holdings_df)
            try:
                ind_df, loc_df = utils_webscrap.aggregate_etf_data(holdings_df, ticker)

                master_industry_list.append(ind_df)
                master_location_list.append(loc_df)
                print(f"✅ {ticker} agg successful")
            except Exception as e:
                print(f"❌ {ticker} agg failed: {e}")
                manual_inspect.append(ticker)

            return None
        return None
    except Exception as e:
        print(f"somthing's wrong...: {e}")
        manual_inspect.append(ticker)


In [ ]:
for i in range(len(records_df)):
    download_url = records_df['download_url'][i]
    referer_url = records_df["product_url"][i]
    ticker = records_df["ticker"][i]
    retrieve_holding_info_from_ishare(download_url, referer_url, ticker)


successfully extracted 298 rows
✅ IQQ0 agg successful

successfully extracted 133 rows
✅ IBCK agg successful

successfully extracted 514 rows
✅ IUSC agg successful

successfully extracted 45 rows
✅ DAXEX agg successful

successfully extracted 163 rows
✅ EUNY agg successful

successfully extracted 129 rows
✅ IUSK agg successful

successfully extracted 598 rows
✅ IUSL agg successful

successfully extracted 68 rows
✅ EXI2 agg successful

successfully extracted 303 rows
✅ EXSI agg successful

successfully extracted 56 rows
✅ EUN2 agg successful

successfully extracted 55 rows
✅ EXW1 agg successful

successfully extracted 33 rows
✅ EXX1 agg successful

successfully extracted 35 rows
✅ IQQA agg successful

successfully extracted 107 rows
✅ IUSZ agg successful

successfully extracted 57 rows
✅ IQQC agg successful

successfully extracted 57 rows
✅ IQQD agg successful

successfully extracted 300 rows
✅ IQQI agg successful

successfully extracted 56 rows
✅ EXS3 agg successful

successfully extr

In [ ]:
# processed_tickers = pd.read_csv('master_industry_table.csv')['Ticker'].unique().tolist()
# manual_inspect = list(set(records_df['ticker'].unique().tolist()) - set(processed_tickers))

In [ ]:
# try again for those in manual_inspect


# selected_indices=records_df[records_df['ticker'].isin(manual_inspect)].index.tolist()


# for idx in selected_indices:
#     download_url = records_df.loc[idx,'download_url']
#     referer_url = records_df.loc[idx,"product_url"]
#     ticker = records_df.loc[idx,"ticker"]
#     print(ticker)
#     print(referer_url)
#     print(download_url)
#     retrieve_holding_info_from_ishare(download_url, referer_url, ticker)


In [ ]:
# get QDVB holdings
qdvb_holdings = pd.read_excel('QDVB.xlsx',header= 7 )

ind_df, loc_df = utils_webscrap.aggregate_etf_data(qdvb_holdings, 'QDVB')

master_industry_list.append(ind_df)
master_location_list.append(loc_df)
print(f"✅ {'QDVB'} agg successful")

In [70]:
# agg to industry/location tables
final_industry_master = pd.concat(master_industry_list, ignore_index=True)
final_location_master = pd.concat(master_location_list, ignore_index=True)
final_industry_master["Sector"] = final_industry_master["Sector"].apply(
    lambda x: x.strip()
)
final_location_master["Location"] = final_location_master["Location"].apply(
    lambda x: x.strip()
)

In [ ]:
# get TER info
import requests
from bs4 import BeautifulSoup
import time
import os
headers = {
    "User-Agent": "Mozilla/5.0"
}
import re

ter_list = []
manual_ter_lst = []

def get_ter(isin):
    url = f"https://www.justetf.com/en/etf-profile.html?isin={isin}"

    rq = requests.get(url, headers=headers)
    if  rq.status_code == 403:
        print(isin)
    html = rq.text
    soup = BeautifulSoup(html, "html.parser")

    # 找到包含 TER 的 label
    labels = soup.find_all(
        "div",
        class_="val bold",
        attrs={"data-testid": "etf-profile-header_ter-value"}
    )
    ter = None
    if len(labels) == 1:
        label = labels[0]
        ter = label.get_text(strip=True).split()[0]
        print("ISIN:",isin,"TER:", ter)
        pattern = r'(\d+\.\d+)(\s*%)'
        ter= float(re.match(pattern,ter).group(1))
        return ({'ISIN':isin,"TER":ter})
    else:
        print(f'ISIN:{isin} not found')
        manual_ter_lst.append(isin)
        return None
        
    
if not os.path.exists('ter_info.csv'):
    for isin in filtered_etf['ISIN'].tolist():
        ter_list.append(get_ter(isin))
        time.sleep(2)

    ter_df = pd.DataFrame(ter_list)
    ter_df.to_csv('ter_info.csv')
else:
    ter_df = pd.read_csv('ter_info.csv')

ISIN: IE00B5BMR087 TER: 0.07%
ISIN: IE00B4L5Y983 TER: 0.20%
ISIN: IE00B52MJD48 TER: 0.48%
ISIN: DE000A0H08D2 TER: 0.51%
ISIN: IE00BKM4GZ66 TER: 0.18%
ISIN: IE00B6R52259 TER: 0.20%
ISIN: IE00B53SZB19 TER: 0.30%
ISIN: IE0031442068 TER: 0.07%
ISIN: IE00BFNM3G45 TER: 0.07%
ISIN: IE00BHZPJ890 TER: 0.07%
ISIN: IE00B3WJKG14 TER: 0.15%
ISIN: IE00B4K48X80 TER: 0.12%
ISIN: IE0005042456 TER: 0.07%
ISIN: IE00BMTX1Y45 TER: 0.05%
ISIN: IE00B1YZSC51 TER: 0.12%
ISIN: DE0002635307 TER: 0.20%
ISIN: IE00B0M62Q58 TER: 0.50%
ISIN: DE0005933931 TER: 0.16%
ISIN: DE0005933956 TER: 0.09%
ISIN: IE00BHZPJ908 TER: 0.07%
ISIN: IE00B3ZW0K18 TER: 0.20%
ISIN: IE00BHZPJ239 TER: 0.18%
ISIN: IE00B0M63177 TER: 0.18%
ISIN: IE00BF4RFH31 TER: 0.35%
ISIN: IE00B4L5YX21 TER: 0.12%
ISIN: IE00B53L3W79 TER: 0.10%
ISIN: IE00B4L5YC18 TER: 0.18%
ISIN: IE00BYX2JD69 TER: 0.20%
ISIN: IE00B53QG562 TER: 0.12%
ISIN: DE000A0F5UF5 TER: 0.30%
ISIN: IE00BFNM3P36 TER: 0.18%
ISIN: IE00BZCQB185 TER: 0.65%
ISIN: IE00B5L8K969 TER: 0.20%
ISIN: IE00

In [79]:
ter_with_ticker_df = pd.merge(filtered_etf[['Ticker','ISIN']],ter_df,how='right',on='ISIN')

In [80]:
final_location_master = pd.merge(final_location_master,ter_with_ticker_df,how='left',on='Ticker')
# save result
final_industry_master.to_csv("master_industry_table.csv", index=False)
final_location_master.to_csv("master_location_table.csv", index=False)
